Importing libraries that will be used.

In [131]:
import pandas as pd
import plotly.express as px

Reading the .csv archive, saving it as df and printing the first 5 rows.

In [132]:
df = pd.read_csv('vehicles_us.csv')
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


Pulling the informations from the .csv file.
We can see that columns year, type, body and acidity has missing values.

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


Here we can see exactly how many missing values we have in each column.

In [134]:
df.isnull().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

A quick summary of key statistical metrics like mean, standard deviation, percentiles, and more.

In [135]:
df.describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


In this step, missing values in critical columns are handled systematically based on logical groupings. Here's the approach for each column:

model_year: Missing values are filled with the median model_year grouped by the model column, ensuring consistency within the same car model. The column is then converted to an integer type.

cylinders: Missing values are filled with the median number of cylinders grouped by the model, ensuring realistic and consistent data for each car model. The column is then converted to an integer type.

odometer: Missing values are filled with the median odometer value grouped by a combination of model_year and model, ensuring a granular and contextually accurate replacement.

is_4wd: Missing values are interpreted as False (0), indicating that the car does not have four-wheel drive. The column is then converted to a boolean type.

paint_color: Missing values are filled with Unknown.

Verification: After filling missing values, the number of null values in each column and the data types of all columns are displayed to ensure the transformations were applied correctly

In [136]:
df['model_year'] = df.groupby('model')['model_year'].transform(lambda x: x.fillna(x.median())).astype(int)
df['cylinders'] = df.groupby('model')['cylinders'].transform(lambda x: x.fillna(x.median())).astype(int)
df['odometer'] = df.groupby(['model_year', 'model'])['odometer'].transform(lambda x: x.fillna(x.median()))
df['is_4wd'] = df['is_4wd'].fillna(0).astype(bool)
df['paint_color'] = df['paint_color'].fillna('Unknown')
print(df.isnull().sum())
print(df.dtypes)

price            0
model_year       0
model            0
condition        0
cylinders        0
fuel             0
odometer        83
transmission     0
type             0
paint_color      0
is_4wd           0
date_posted      0
days_listed      0
dtype: int64
price             int64
model_year        int32
model            object
condition        object
cylinders         int32
fuel             object
odometer        float64
transmission     object
type             object
paint_color      object
is_4wd             bool
date_posted      object
days_listed       int64
dtype: object


Model Year (model_year) by Cylinders (cylinders): This scatter plot illustrates how the number of cylinders has evolved across different model years. We can observe that almost all cars manufactured before 1980 feature either 6 or 8 cylinders. From that point onward, a broader range of vehicle categories emerged, with models spanning from 3 to 12 cylinders.

In [137]:
fig1 = px.scatter(df, x='model_year', y='cylinders', title='Year Model by Cylinders',
                  labels={'model_year': 'Year Model', 'cylinders': 'Cylinders'})
fig1.show()

Distribution of Model Year: This histogram visualizes the distribution of model years, providing insights into which years are most prevalent in the dataset. It reveals that only a small number of classic cars exist from before 1980. In contrast, the majority of vehicles in the dataset are concentrated around the 2010-2014 period.


In [138]:
fig2 = px.histogram(df, x='model_year', title='Distribution of Model Year', nbins=30)
fig2.update_xaxes(title='Model Year')
fig2.update_yaxes(title='Frequency')
fig2.show()

Distribution of Number of Cylinders: This histogram displays the distribution of cars based on the number of cylinders, allowing us to observe whether vehicles with fewer or more cylinders are more common. We can see that most cars have between 4 and 8 cylinders, with fewer vehicles featuring more extreme values, such as 3 or 12 cylinders.

In [139]:
fig3 = px.histogram(df, x='cylinders', title='Distribution of Number of Cylinders', nbins=15)
fig3.update_xaxes(title='Number of Cylinders')
fig3.update_yaxes(title='Frequency')
fig3.show()